В этом ноутбуке продемонстрирован пример инференса модели на примере валидационной выборки из 10 случайных запросов

Валидационная выборка состоит из полей:
1. Product Name - название продукта
2. clean_desc - описание товара от 10 слов
3. search_text - дефолтный текст запроса типа "soda" и тд. Сделано для всех 4к строк
4. abstract_search_text - абстрактный запрос вроде "Rich baking ingredients for traditional recipes", сделано для 20% датафрейма, остальное Nan

У каждого товара есть 1 или 2 запроса. Для валидации мы к каждому запросу из тест сета моделью подберём 20 релевантных товаров с помощью модели

#### **инсталлы**

In [6]:
import torch
import transformers
import pandas as pd
import numpy as np
from tqdm import tqdm
from scipy.spatial.distance import cdist
from transformers import AutoTokenizer, DistilBertModel

In [20]:
def seed_everything(seed = 42, torch_deterministic = True):
  # TRY NOT TO MODIFY: seeding
  # random.seed(seed)
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.backends.cudnn.deterministic = torch_deterministic

seed_everything(42)

#### **загрузка и подготовка данных**

In [11]:
df_val = pd.read_csv("/content/items_with_search_text.csv")
df_val = df_val[["Uniq Id", "Product Name", "clean_desc", "search_text", "abstract_search_text"]]
df_val.head()

,Uniq Id,Product Name,clean_desc,search_text,abstract_search_text
0,019b67ef7f01103d8fb0a53e4c36daa7,"La Costena Chipotle Peppers, 7 OZ (Pack of 12)","la costena chipotle peppers, 7 oz (pack of 12)...",chipotle peppers,Mexican spice essentials
1,e4fab4b6f41eac02d22b421818c8f080,(6 Boxes) Twinings of London Nightly Calm Gree...,enojy one of your favorite tea flavors with th...,green tea,Soothing bedtime teas
2,992c11a1b238eae6a2fabf22c74de5c2,"Tiny White Mighty Mints (16 oz, ZIN: 525424) -...","tiny white mighty mints (16 oz, zin: 525424): ...",mints,Breath freshening essentials
3,290970606fd82ac39534aa32b8b3b149,SweetGourmet Narrow Sesame Sticks | Lightly Sa...,"narrow sesame sticks are fresh, crunchy and li...",sesame sticks,Savory snack ideas
4,21fee4394b9cc53bb6ddbe4235506a5c,"Health Warrior Chia Bar, Acai Berry, 25 G, Pac...","health warrior chia bar, acai berry is packed ...",chia bar,Healthy snack alternatives


#### **подготовка модели**

In [12]:
# модель для проверки
class Model:
    def __init__(self, product_DB: pd.DataFrame, to_set_up: bool = False, path_to_vector_BD: str = None):
        self.model = None
        self.tokenizer = None

        self.vector_DB = None
        if path_to_vector_BD:
            print("Downloading Vector BD")
            self.vector_DB = np.load(path_to_vector_BD)

        self.products_DB = product_DB.sort_values(by = "Uniq Id")


        if to_set_up:
            self.set_up()

    def set_up(self, max_position_embeddings = 512):
        self.tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
        self.model = DistilBertModel.from_pretrained("distilbert-base-uncased", max_position_embeddings = max_position_embeddings)

    def calc_embeding(self, X: str):
        if not self.model:
            print("U should set_up() nodel first!")
            return
        inputs = self.tokenizer(X[:512], return_tensors="pt")
        return self.model(**inputs).last_hidden_state[:,-1,:].detach().numpy()


    def form_vector_DB(self):
        if self.vector_DB is not None:
            print("Vector DB already exists!")
            return

        self.vector_DB = np.stack(
            tuple( self.calc_embeding(row["Description"]) for _, row in tqdm(self.products_DB.sort_values(by = "Uniq Id").iterrows(), total = len(self.products_DB))
            )
        )

    def get_n_rank(self, query, n = 10):
        if self.vector_DB is None:
            print("No Vector DB exists!")
            return

        if not self.model:
            print("U should set_up() nodel first!")
            return

        query_v = self.calc_embeding(query)
        top_indx = cdist(query_v, self.vector_DB, 'cosine')[0].argsort()[:n] # top most simular (with smallar distance)
        out_uniq_id = self.products_DB.sort_values(by = "Uniq Id").iloc[top_indx]["Uniq Id"]
        return out_uniq_id

In [13]:
# инициализируем модель для инференса
inference_model = Model(df_val, path_to_vector_BD = "vector_BD.npy", to_set_up = True)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

#### **инференс на тестовых запросах**

In [91]:
# выберем k тестовых стандартных(не абстрактных) запросов случайно из выборки
seed_everything(42)
k = 100
random_indices = np.random.randint(0, len(df_val), size=k)

test_queries = df_val.loc[random_indices]
#test_products = df_val.loc[random_indices][["Uniq Id", "Product Name", "clean_desc"]]
test_queries

,Uniq Id,Product Name,clean_desc,search_text,abstract_search_text
3174,49211ae8276d458310dd06f536653fb8,Yumbutter Superfood Almond Butter - Case Of 6 ...,yumbutter superfood almond butter - case of 6 ...,Almond butter,NaN
3507,48f177144587bb50f2298a14fdc02b86,"Setton Farms Omega-3 Mix, 18 Oz",setton farms mixes are created with unique ble...,omega-3 mix,NaN
860,e84cb9e22be2bdf90e1e01b0a273c263,"Sanpellegrino Orange Sparkling Fruit Beverage,...","inspired by homemade italian aranciata, sanpel...",Sparkling beverage,NaN
1294,325db3ffed5c3f15b53530920433bfa0,Lily Of The Desert Organic Aloe Vera Juice Who...,lily of the desert organic aloe vera juice who...,Aloe vera juice,NaN
1130,cf37eea3b3566ea9e32b3f305875cbb3,Raymundos No Added Sugar Strawberry Fruit & Ge...,fruit & gel 7.5oz raymundos straw frt & gel 7.5oz,Strawberry snack,NaN
...,...,...,...,...,...
2613,6fcd4ee41ec44eb339f48f3ad8e3146e,Hakka Manual U-shape Sausage Clipper Clipping ...,hakka manual u-shape sausage clipper clipping ...,sausage maker,NaN
1129,7af8e63f8112eea9c0cf1d6ea9225c80,"Baking Soda (16 oz, ZIN: 524710)","baking soda (16 oz, zin: 524710): our baking s...",Baking Ingredient,NaN
3843,16a67beab72bc5a0121bf5cd125f3792,"Pack of 2 - Cinnamon Churro Marshmallows, 4.5 oz","organic cane sugar, water, tapioca syrup, gela...",Cinnamon marshmallows,NaN
3893,1903ded19fd20ff283c1f925042596bb,Bar Harbor - Clam Chowder - Ready To Serve - C...,bar harbor - clam chowder - ready to serve - c...,Clam chowder,NaN


In [92]:
# проинферим модель на запросах валидации
top_n = 100
model_output = dict()
for row in test_queries.iterrows():
  product_ids = inference_model.get_n_rank(row[1]["search_text"], top_n)
  model_output[row[1]["Uniq Id"]] = product_ids

#### **метрики**

на первом этапе посчитаем hitrate@k и как итоговое качество - долю запросов, на которых hitrate@k (k пока какое-то большое) = 1

In [101]:
def hitrate_at_k(model_output, test_queries, k=100):
  hitrates = 0
  # проитерируемся по всем предсказаниям
  for uniq_id in model_output.keys():
    row = test_queries[test_queries["Uniq Id"] == uniq_id]
    predicted_product_ids = model_output[uniq_id]
    # проверка, есть ли исходный товар в предсказанных товарах по запросу для этого товара
    if uniq_id in set(model_output[uniq_id]):
      hitrates += 1
  return hitrates

In [104]:
print(hitrate_at_k(model_output, test_queries))

4


интерпретация: на 100 тестовых запросах у 4 запросов в топ100 есть товар, по которому генерился запрос.

In [109]:
# проитерируемся по выходам модели и посмотрим на запросы и то, что модель выдала на них
for uniq_id in model_output.keys():
  row = test_queries[test_queries["Uniq Id"] == uniq_id]
  predicted_product_ids = model_output[uniq_id]

  print(f"query was: {row['search_text']} \ncorrect product is: {row['Product Name']}")
  print(f"predicted products are:\n {df_val[df_val['Uniq Id'].isin(predicted_product_ids)]['Product Name'].values[:20]}\n\n")

query was: 3174    Almond butter
Name: search_text, dtype: object 
correct product is: 3174    Yumbutter Superfood Almond Butter - Case Of 6 ...
Name: Product Name, dtype: object
predicted products are:
 ['Picamas Green Hot sauce 3.52 oz - Salsa verde picante (Pack of 24)'
 'Rice Cake Daifuku Mochi with Red Bean Paste Filling 8 peice per Pack (Red & White, 2 Pack)'
 'Sechlers Hungarian Sweet-Mild Pepper Relish, 16 oz (Pack of 6)'
 'Popcorn Indiana Aged Wisconsin White Cheddar Crispy Popcorn 3.5oz (PACK OF 6)'
 'Corn Starch (8 oz, ZIN: 526021) - 3-Pack'
 'Larabar Gluten Free Coconut Cream Pie Fruit and Nut Bars 5 ct, 8.5 oz'
 'Sahale Snacks Glazed Mixes, Classic Fruit Nut, 1.5 oz, Case of 9'
 'Froozer Orange Zinger 6 Count' 'Great Value Honey Barbeque Corn Chips'
 '(6 Pack) Swiss Miss Rich Chocolate Hot Cocoa Mix Envelope, 8 count'
 'Maple Hill Creamery Organic 2% Reduced-Fat Unflavored Milk, Half Gallon'
 'Dip & Dessert Mix - 20 Oz Tub (Strawberry)'
 '(3 Pack) MCP Premium Fruit Pectin,

На тестовых запросах по топ 100 продуктов на каждый видим, что необученный берт пока не может подтянуть товар, по которому создавался запрос, однако в выдаче есть релевантные товары, например, для запроса Espresso coffee в выдаче есть другие виды кофе, а на запрос Strawberry snack выдаются десерты/чай. При этом есть товары, которые выдаются высоко практически на каждый продукт, отдалённо похожий на них - например, Rice Cake Daifuku Mochi with Red Bean Paste Filling 8 peice per Pack (Red & White, 2 Pack). Возможно модель триггерится на большое разнообразие общих слов в названии/описании и поэтому часто выдаёт этот товар.

Для дальнейшей работы по улучшению качества модели можно выделить следующие варианты:
- взять модель побольше, посильнее, потолще и возможно предобученную на доменах ретейл/ecom
- попробовать другой датасет
- добавить препроцессинг описания товаров/запросов
- подготовить качественный валидационный датасет
- самостоятельно дообучить модель на домене